# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

Airline ticket by using multiple tools

Because llama or others doesn't have openai style tool_call structure

1) We change system message with few shot example. It says if it is a function call, response in this structure.
2) In the chat(message, history) function first we try to understand if it is a function call. If it is not it returns fisrt response, if it is a function call it call function and change the response

In [1]:
# imports


import json
import gradio as gr
import requests


In [2]:
# Constants
import ollama

OLLAMA_API = "http://localhost:11434/api/chat"
MODEL = "llama3.2"


In [3]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [4]:
# system_message = "You are a helpful assistant for an Airline called FlightAI. "
# system_message += "Give short, courteous answers, no more than 1 sentence. "
# system_message += "Always be accurate. If you don't know the answer, say so."

In [5]:
# # closed source models like llama cant use

# system_message += """
# You are a helpful assistant that can call tools by returning a JSON object.

# If the user asks a question that requires external data (e.g., ticket price or places to see in a city), respond with a JSON object in this format:

# {
#   "function": "FUNCTION_NAME",
#   "arguments": {
#     "destination_city": "CITY_NAME"
#   }
# }

# Supported functions:
# 1. get_ticket_price(destination_city)
# 2. get_places_to_see(destination_city)

# Only return the JSON, nothing else. Do not answer the question yourself.
# """

In [6]:
system_message = """
You are a helpful assistant for an airline called FlightAI.

If the user asks something that needs external data, call a function by returning a JSON object like:

{
  "function": "get_ticket_price",
  "arguments": {
    "destination_city": "London"
  }
}

After receiving a tool response (from role: tool), you MUST give a short final answer in plain language, in ONE sentence. Do NOT return JSON after that.

Supported functions:
- get_ticket_price(destination_city)
- get_places_to_see(destination_city)
"""

## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

There  will be 2 tools at the beginnig. 
1: Will tell the ticket price
2. Will tell the places to see

In [7]:
# 1: Ticket price function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}") # this way we can see this function called in output
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown") # ticket_prices'te bulamazsa Unknown yazıyor
    # return ticket_prices.get(city) # bu boş dönüyor bulamazsa

In [8]:
# get_ticket_price("Berlin")

In [9]:
# There's a particular dictionary structure that's required to describe our function:
# 1st function for first tool

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price,\
                    for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [10]:
# 2: Places to see function

places_to_see = {"london": "London Eye", "paris": "Eyfel", "tokyo": "Tokyo Drift", "berlin": "I don't know", \
                "istanbul": "Şükrü Saraçoğlu, Fenerbahçe Stadium", "bayburt": "Ortugu. \
                This place is where onur has born. Everybody should see there."}

def get_places_to_see(destination_city): 
    print(f"Tool get_places_to_see called for {destination_city}") # this way we can see this function called in output
    city = destination_city.lower()
    return places_to_see.get(city, "Bimiyorum ki valla Onur'a sor")

In [11]:
# get_places_to_see("Bayburt")

In [12]:
# get_places_to_see("Ankara")

In [13]:
# There's a particular dictionary structure that's required to describe our function:
# 2nd function for second tool

place_function = {
    "name": "get_places_to_see",
    "description": "Get the place to see in the destination city. \
                    Call this whenever you need to know the place to visit,\
                    for example when a customer asks 'Where can I visit in this city' or 'Where should I see in this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [14]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}, 
        {"type": "function", "function": place_function}]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [15]:
# def chat(message, history):
#     messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
#     response = ollama.chat(model=MODEL, messages=messages_for(website), tools=tools)
#     print(f"finish reason {response.choices[0].finish_reason}")
#     print(f"Response {response.choices[0].message.content}")

#     if response.choices[0].finish_reason=="tool_calls":
#         message = response.choices[0].message
#         print(f" \n, Tool call message: {message}, \n")
#         response, city = handle_tool_call(message)
#         messages.append(message)
#         messages.append(response)
#         response = openai.chat.completions.create(model=MODEL, messages=messages)
    
#     return response.choices[0].message.content

In [16]:


def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    response = ollama.chat(model=MODEL, messages=messages)
    print(f"Response: {response['message']['content']}")

    # Try to parse the JSON to simulate tool call
    try:
        print("a")
        tool_call_data = json.loads(response['message']['content'])
        print("b")
        function_name = tool_call_data.get("function")
        arguments = tool_call_data.get("arguments", {})
        destination_city = arguments.get("destination_city")
        print("c")
        print(f" çağrılan fonksiyon: {function_name}")

        if function_name in ["get_ticket_price", "get_places_to_see"]:
            tool_response = handle_tool_call(function_name, destination_city)
            messages.append({"role": "assistant", "content": response['message']['content']})
            messages.append(tool_response)

            # Get final answer after tool call
            response = ollama.chat(model=MODEL, messages=messages)
            return response['message']['content']
    except Exception as e:
        print(f"No tool call detected or JSON parse error: {e}")

    return response['message']['content']

In [17]:
# # We have to write that function handle_tool_call:

# def handle_tool_call(message):
#     tool_call = message.tool_calls[0] # message already has which tool to call
#     arguments = json.loads(tool_call.function.arguments) # because it is in json format
#     city = arguments.get('destination_city')
#     function_name = tool_call.function.name # this is just a string, look at message(call tool message)
    
#     # price = get_ticket_price(city) we dont know which function to use
#     func = 0
#     if function_name == "get_ticket_price":
#         price = get_ticket_price(city)
#         func = 1
#     elif function_name == "get_places_to_see":
#         place = get_places_to_see(city)
#         func = 2
        
#     if func == 1:
#         response = {
#         "role": "tool",
#         "content": json.dumps({"destination_city": city,"price": price}), # destination_city tool call messagının içinde var
#         "tool_call_id": tool_call.id
#         }
#     elif func == 2:
#         response = {
#         "role": "tool",
#         "content": json.dumps({"destination_city": city,"place": place}), # destination_city tool call messagının içinde var
#         "tool_call_id": tool_call.id
#         }
        
#     return response, city

In [18]:
def handle_tool_call(function_name, city):
    print(f"Tool {function_name} called for {city}")
    city_lower = city.lower()
    
    if function_name == "get_ticket_price":
        print("1")
        price = get_ticket_price(city)
        return {
            "role": "tool",
            "content": json.dumps({"destination_city": city, "price": price})
        }
    
    elif function_name == "get_places_to_see":
        print("2")
        place = get_places_to_see(city)
        return {
            "role": "tool",
            "content": json.dumps({"destination_city": city, "place": place})
        }

    return {"role": "tool", "content": json.dumps({"error": "Unknown function"})}

In [19]:
gr.ChatInterface(fn=chat, type="messages").launch()

# call id'de hata alıyor. şehir ismi verip taskı tam anlatmazsan karıştırıyor.

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Response: Hello! How can I assist you with your FlightAI query today?
a
No tool call detected or JSON parse error: Expecting value: line 1 column 1 (char 0)
